studia query [qua](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)
    * continua a studiare link al tutorial "Building high-quality..." (da "Example queries & rules")
        * finora interessante solo:
            * link ad [annotations](https://developer.twitter.com/en/docs/twitter-api/annotations/overview)
            * In the new Twitter API (recent search and filtered stream), AND is applied before OR

verifica quello che dice chatGPT (i like invece credo li abbia solo il tweet originale):
"When you search for tweets using Tweepy, you will get the original tweet and any retweeted versions of that tweet as separate results. Retweets are considered to be separate tweets from the original, so you will receive multiple results for the same content if it has been retweeted multiple times. To avoid receiving duplicate results, you can use the .dedup() method of the Cursor object to remove duplicates. This will only return one result for each unique tweet, even if it has been retweeted multiple times."

In [4]:
bearer_token = "..." #TODO

In [74]:
from pprint import pprint
import tweepy
client = tweepy.Client(bearer_token)
response = client.search_recent_tweets( #TODO adjust (filtra food con
                                        #context:<domain_id>.<entity_id> (asterischi?),
                                        #lang:en ecc.)
    query='context:152.* lang:en', #all English Tweets about the domain "Food"
    expansions=[
        'author_id'],
    tweet_fields=[
        'public_metrics'])
for tweet in response.data:
    pprint(tweet.data)
    print('\n')

{'author_id': '1221233100216205312',
 'edit_history_tweet_ids': ['1610389084190425089'],
 'id': '1610389084190425089',
 'public_metrics': {'like_count': 0,
                    'quote_count': 0,
                    'reply_count': 0,
                    'retweet_count': 3},
 'text': 'RT @PompeyOnSixers: As expected, Joel Embiid is the Eastern '
         'Conference Player of the Month for December. He becomes the first '
         '#Sixers playe…'}


{'author_id': '1432752982265896966',
 'edit_history_tweet_ids': ['1610389084022743041'],
 'id': '1610389084022743041',
 'public_metrics': {'like_count': 0,
                    'quote_count': 0,
                    'reply_count': 0,
                    'retweet_count': 16},
 'text': 'RT @pdpatt: Just revealed 4/20 of my @savagenationnft. Don’t sleep '
         'on this project. Art is too fire! https://t.co/GmGATFxevV'}


{'author_id': '747425638391242752',
 'edit_history_tweet_ids': ['1610389083838386176'],
 'id': '1610389083838386176',
 'pub

In [ ]:
#like sempre a zero se è un retweet. Vedi:
https://twittercommunity.com/t/tweets-having-likes-more-than-zero-always-shows-zero-while-on-twitter-it-shows-more-than-zero-likes-it-only-shows-retweets-counts-accurate-please-tell-me-how-to-get-the-correct-like-count-as-well-i-am-using-twitter-v2-search-api/160840/4